In [1]:
!pip install numpy==1.16.1

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import keras
from keras.datasets import imdb
import numpy as np
import matplotlib as plt


/home/viettq/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
(train_data, train_labels),(test_data, test_labels) = imdb.load_data(num_words=10000)

In [5]:
print(train_data.shape)
print(train_labels.shape)

(25000,)
(25000,)


In [6]:
# EDA
def vectorize_sequences(sequences , dim = 10000):
    results = np.zeros((len(sequences), dim))
    for i,sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [7]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)



In [23]:
#Architect
from keras import models
from keras import layers

def net_model(x_train, x_labels, y_test, y_labels):
    model = models.Sequential()
    model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])
    
    model.fit(x_train,
              x_labels,
              epochs = 5,
              batch_size = 32,
              validation_data = (y_test, y_labels)
              )
    
    
    return model

In [24]:
#k-fold
fold = 4
k = len(x_train)//fold
validation_scores = []
for i in range(fold):
    x_val = x_train[i*k:k*(i+1)]
    y_val = train_labels[i*k:k*(i+1)]
    
    x_val_pratial = np.concatenate([x_train[:i*k], x_train[k*(i+1):]]) 
    y_val_pratial = np.concatenate([train_labels[:i*k], train_labels[k*(i+1):]]) 
    network = net_model(x_val_pratial,y_val_pratial,x_val,y_val)
    validation_score = network.evaluate(x_test,test_labels)
    validation_scores.append(validation_score)

Train on 18750 samples, validate on 6250 samples
Epoch 1/5
18750/18750 [==============================] - 10s 507us/step - loss: 0.3305 - acc: 0.8654 - val_loss: 0.2690 - val_acc: 0.8944
Epoch 2/5
18750/18750 [==============================] - 5s 269us/step - loss: 0.2057 - acc: 0.9226 - val_loss: 0.3081 - val_acc: 0.8827
Epoch 3/5
18750/18750 [==============================] - 5s 270us/step - loss: 0.1677 - acc: 0.9375 - val_loss: 0.3032 - val_acc: 0.8901
Epoch 4/5
18750/18750 [==============================] - 5s 272us/step - loss: 0.1442 - acc: 0.9466 - val_loss: 0.3225 - val_acc: 0.8890
Epoch 5/5
25000/25000 [==============================] - 16s 630us/step
Train on 18750 samples, validate on 6250 samples
Epoch 1/5
18750/18750 [==============================] - 8s 447us/step - loss: 0.3326 - acc: 0.8649 - val_loss: 0.2843 - val_acc: 0.8862
Epoch 2/5
18750/18750 [==============================] - 6s 303us/step - loss: 0.2073 - acc: 0.9209 - val_loss: 0.2880 - val_acc: 0.8914
Epoch 3

In [25]:
validation_scores

[[0.40579821220874784, 0.865],
 [0.41164810635089877, 0.87032],
 [0.40816759532928465, 0.86712],
 [0.4473195570421219, 0.86344]]